In [1]:
#
#
#

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

def get_url(s):
    base_url = f'https://docs.google.com/spreadsheets/d/1pJ-jeDmVb69yCgUbi0DsRT-HW8jfJlZBh6ehyelOaOw/export?gid=%d&format=csv'

    if s == 'data/reloading_data - powder burning rate chart.csv':
        return base_url % 0
    if s == 'data/reloading_data - CATRIDGES.csv':
        return base_url % 1775574147
    if s == 'data/reloading_data - 9mm parabellum.csv':
        return base_url % 1119510158
    if s == 'data/reloading_data - 38 special.csv':
        return base_url % 1030409937
    if s =='data/reloading_data - 357 magnum.csv':
        return base_url % 847084052
    raise Exception("Unknown filename: " + s)

In [10]:
# data/reloading_data - CATRIDGES.csv
cartridges = pd.read_csv(get_url('data/reloading_data - CATRIDGES.csv'))
print(cartridges)

cartridges_to_process = cartridges.loc[:, ['title', 'volume', 'diameter']].to_numpy()
print(cartridges_to_process)

# reloading_data - powder burning rate chart
powder_rates = pd.read_csv(get_url('data/reloading_data - powder burning rate chart.csv'))
print(powder_rates.head())

             title  volume  diameter
0   9mm parabellum    0.74     0.356
1       38 special    1.30     0.358
2       357 magnum    1.36     0.358
3  7.62x54 russian    3.47     0.310
[['9mm parabellum' 0.74 0.356]
 ['38 special' 1.3 0.358]
 ['357 magnum' 1.36 0.358]
 ['7.62x54 russian' 3.47 0.31]]
        powder  rate note
0   A NITRO100     1  NaN
1       v-N310     1  NaN
2     VEC BA10     2  NaN
3     ALLNT E3     2  NaN
4  WIN AA LITE     2  NaN


In [14]:
def process_data_file(c, v, a):
    filename = f'data/reloading_data - %s.csv' % c
    try:
        d = pd.read_csv(get_url(filename))
        d['cartridge_caliber'] = c
        d['cartridge_volume'] = v
        d['cartridge_diameter'] = a
        return d
    except:
        print("ERROR: Cannot process:", filename)
        return None

frames = [ process_data_file(c, v, a) for c, v, a in cartridges_to_process ]
data = pd.concat(frames, ignore_index=True)

# print(data.head())  # DEBUG
print('data.shape', data.shape)

ERROR: Cannot process: data/reloading_data - 7.62x54 russian.csv
data.shape (216, 16)


In [13]:
data[~ np.isnan(data.press_psi)]

,powder,start,volume cc,auto disk,lee dipper,velocity,never exceed,velocity max,press,units,min oal,mm,press_psi,cartridge_caliber,cartridge_volume,cartridge_diameter
33,UNIVERSAL,4.0,NaN,NaN,NaN,1034.0,4.5,1124.0,31300.0,CUP,1.1,27.94,29361.0,9mm parabellum,0.74,0.356
39,CLAYS,3.0,NaN,NaN,NaN,954.0,3.4,1039.0,31000.0,CUP,1.1,27.94,28908.0,9mm parabellum,0.74,0.356
